In [14]:
#Importar Variables
import pandas as pd
import os
import numpy as np

#Cargar el dataset
main_path="drive/MyDrive/Dataset_MP"
name= "df_limpio.csv"
full_path=os.path.join(main_path,name)

In [15]:
df=pd.read_csv(full_path)
df.info()

/tmp/ipython-input-15-106955314.py:1: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(full_path)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5714008 entries, 0 to 5714007
Data columns (total 12 columns):
 #   Column               Dtype  
---  ------               -----  
 0   MONTH                int64  
 1   DAY                  int64  
 2   DAY_OF_WEEK          int64  
 3   AIRLINE              object 
 4   FLIGHT_NUMBER        int64  
 5   ORIGIN_AIRPORT       object 
 6   DESTINATION_AIRPORT  object 
 7   SCHEDULED_DEPARTURE  int64  
 8   SCHEDULED_TIME       float64
 9   DISTANCE             int64  
 10  SCHEDULED_ARRIVAL    int64  
 11  DELAYED              int64  
dtypes: float64(1), int64(8), object(3)
memory usage: 523.1+ MB


In [16]:
#Reducir cardinalidad de aeropuertos
top_origen = df['ORIGIN_AIRPORT'].value_counts().nlargest(20).index
df['ORIGIN_AIRPORT'] = df['ORIGIN_AIRPORT'].apply(lambda x: x if x in top_origen else 'OTHER')

top_dest = df['DESTINATION_AIRPORT'].value_counts().nlargest(20).index
df['DESTINATION_AIRPORT'] = df['DESTINATION_AIRPORT'].apply(lambda x: x if x in top_dest else 'OTHER')

#Eliminar FLIGHT_NUMBER (poco útil y muy cardinal)
df.drop(columns=['FLIGHT_NUMBER'], inplace=True)

In [17]:
for col in ['AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT']:
    df[col] = df[col].fillna('missing').astype(str)

In [18]:
#Codificación trigonométrica de variables cíclicas
#Es para representar la hora de forma cíclica, porque el día reinicia cada 24 h y el modelo no lo entiende si se lo das como número normal.
def transformar_hora(df, columna):
    #Convertir a string y rellenar con ceros a la izquierda para formato 4 dígitos
    df[columna] = df[columna].astype(str).str.zfill(4)

    #Calcular minutos desde medianoche directamente
    minutos_desde_medianoche = df[columna].str[:2].astype(int) * 60 + df[columna].str[2:].astype(int)

    # Variables trigonométricas para capturar la periodicidad del tiempo (24h = 1440 minutos)
    df[columna + '_sin'] = np.sin(2 * np.pi * minutos_desde_medianoche / 1440)
    df[columna + '_cos'] = np.cos(2 * np.pi * minutos_desde_medianoche / 1440)
    return df

# Aplicar la transformación a ambas columnas
df = transformar_hora(df, 'SCHEDULED_DEPARTURE')
df = transformar_hora(df, 'SCHEDULED_ARRIVAL')

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5714008 entries, 0 to 5714007
Data columns (total 15 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   MONTH                    int64  
 1   DAY                      int64  
 2   DAY_OF_WEEK              int64  
 3   AIRLINE                  object 
 4   ORIGIN_AIRPORT           object 
 5   DESTINATION_AIRPORT      object 
 6   SCHEDULED_DEPARTURE      object 
 7   SCHEDULED_TIME           float64
 8   DISTANCE                 int64  
 9   SCHEDULED_ARRIVAL        object 
 10  DELAYED                  int64  
 11  SCHEDULED_DEPARTURE_sin  float64
 12  SCHEDULED_DEPARTURE_cos  float64
 13  SCHEDULED_ARRIVAL_sin    float64
 14  SCHEDULED_ARRIVAL_cos    float64
dtypes: float64(5), int64(5), object(5)
memory usage: 653.9+ MB


In [20]:
# Eliminar columnas originales que ya no se usarán
df.drop(columns=['SCHEDULED_DEPARTURE', 'SCHEDULED_ARRIVAL'], inplace=True)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5714008 entries, 0 to 5714007
Data columns (total 13 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   MONTH                    int64  
 1   DAY                      int64  
 2   DAY_OF_WEEK              int64  
 3   AIRLINE                  object 
 4   ORIGIN_AIRPORT           object 
 5   DESTINATION_AIRPORT      object 
 6   SCHEDULED_TIME           float64
 7   DISTANCE                 int64  
 8   DELAYED                  int64  
 9   SCHEDULED_DEPARTURE_sin  float64
 10  SCHEDULED_DEPARTURE_cos  float64
 11  SCHEDULED_ARRIVAL_sin    float64
 12  SCHEDULED_ARRIVAL_cos    float64
dtypes: float64(5), int64(5), object(3)
memory usage: 566.7+ MB


In [22]:
df.describe()

,MONTH,DAY,DAY_OF_WEEK,SCHEDULED_TIME,DISTANCE,DELAYED,SCHEDULED_DEPARTURE_sin,SCHEDULED_DEPARTURE_cos,SCHEDULED_ARRIVAL_sin,SCHEDULED_ARRIVAL_cos
count,5.714008e+06,5.714008e+06,5.714008e+06,5.714008e+06,5.714008e+06,5.714008e+06,5.714008e+06,5.714008e+06,5.714008e+06,5.714008e+06
mean,6.547799e+00,1.570759e+01,3.932643e+00,1.418940e+02,8.244569e+02,1.861109e-01,-1.288751e-01,-3.584280e-01,-3.000328e-01,-2.230113e-01
std,3.397421e+00,8.774394e+00,1.985967e+00,7.531400e+01,6.086620e+02,3.891961e-01,7.361489e-01,5.594689e-01,6.272467e-01,6.832335e-01
min,1.000000e+00,1.000000e+00,1.000000e+00,1.800000e+01,3.100000e+01,0.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00
25%,4.000000e+00,8.000000e+00,2.000000e+00,8.500000e+01,3.730000e+02,0.000000e+00,-8.549119e-01,-8.660254e-01,-8.660254e-01,-8.549119e-01
50%,7.000000e+00,1.600000e+01,4.000000e+00,1.230000e+02,6.500000e+02,0.000000e+00,-3.090170e-01,-5.000000e-01,-4.809888e-01,-4.265687e-01
75%,9.000000e+00,2.300000e+01,6.000000e+00,1.740000e+02,1.065000e+03,0.000000e+00,6.427876e-01,6.123234e-17,1.822355e-01,3.826834e-01
max,1.200000e+01,3.100000e+01,7.000000e+00,7.180000e+02,4.983000e+03,1.000000e+00,1.000000e+00,9.999905e-01,1.000000e+00,1.000000e+00


In [23]:
df.head()

,MONTH,DAY,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_TIME,DISTANCE,DELAYED,SCHEDULED_DEPARTURE_sin,SCHEDULED_DEPARTURE_cos,SCHEDULED_ARRIVAL_sin,SCHEDULED_ARRIVAL_cos
0,1,1,4,AS,OTHER,SEA,205.0,1448,0,0.021815,0.999762,0.923880,0.382683
1,1,1,4,AA,LAX,OTHER,280.0,2330,0,0.043619,0.999048,0.887011,-0.461749
2,1,1,4,US,SFO,CLT,286.0,2296,0,0.087156,0.996195,0.852640,-0.522499
3,1,1,4,AA,LAX,OTHER,285.0,2342,0,0.087156,0.996195,0.854912,-0.518773
4,1,1,4,AS,SEA,OTHER,235.0,1448,0,0.108867,0.994056,0.766044,0.642788


In [24]:
df.shape

(5714008, 13)

In [25]:
df.isnull().sum()

,0
MONTH,0
DAY,0
DAY_OF_WEEK,0
AIRLINE,0
ORIGIN_AIRPORT,0
DESTINATION_AIRPORT,0
SCHEDULED_TIME,0
DISTANCE,0
DELAYED,0
SCHEDULED_DEPARTURE_sin,0


In [26]:
df.to_csv('df_final.csv', index=False)